In [1]:
import pandas as pd
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pprint import pprint
from pathlib import Path

In [2]:
class ScientificFieldInfo(BaseModel):
    code: int
    field_name: str
    domain_name: str
    eklektores_idiou: list[int] = Field(default_factory=list)
    eklektores_synafous: list[int] = Field(default_factory=list)

In [3]:
@dataclass
class ScientificField:
    info: ScientificFieldInfo
    table: pd.DataFrame = None

    def export_ino_to_json(self, filepath: Path):
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(self.info.model_dump_json(indent=4, ensure_ascii=
    
False))

In [4]:
def scientific_field_info_from_excel(
   excel_file: pd.ExcelFile,
    field_code: int
    ) -> ScientificFieldInfo:

    df_sheet = pd.read_excel(excel_file, sheet_name=str(field_code), header=None)
    # Get value from cell F6 (row 5, column 5 in 0-indexed)
    if len(df_sheet) > 5 and len(df_sheet.columns) > 10:
        _code = field_code
        _field = df_sheet.iat[5, 5]      # F6
        _domain = df_sheet.iat[5, 10]    # K6

    sfi = ScientificFieldInfo(
    code=field_code, 
    field_name=_field, 
    domain_name=_domain)

    return sfi

In [5]:
def scientific_field_data_from_excel(
   excel_file: pd.ExcelFile,
    field_code: int
    ) -> pd.DataFrame:

    return pd.read_excel(excel_file, sheet_name=str(field_code), header=8, usecols='A:L')


In [6]:
def eklektores_from_dataframe(
    df: pd.DataFrame
    ) -> tuple[list[int], list[int]]:

    idiou_list = df[df['Χαρακτη-ρισμός'] == 'ΙΔΙΟΥ']['Κωδικός Χρήστη'].tolist()
    synafous_list = df[df['Χαρακτη-ρισμός'] == 'ΣΥΝΑΦΟΥΣ']['Κωδικός Χρήστη'].tolist()

    return idiou_list, synafous_list

In [7]:
def get_full_scientific_fields_from_excel(excel_filename: str) -> dict[int, ScientificField]:

    fields: dict[int, ScientificField] = {}

    excel_file: pd.ExcelFile = pd.ExcelFile(excel_filename)

    field_codes: list[int] = [int(x) for x in excel_file.sheet_names]

    for field_code in field_codes:
        sfi = scientific_field_info_from_excel(excel_file, field_code)
        table = scientific_field_data_from_excel(excel_file, field_code)

        sfi.eklektores_idiou, sfi.eklektores_synafous = eklektores_from_dataframe(table)

        sf = ScientificField(info=sfi, table=table)

        fields[field_code] = sf

    return fields

## Load data

### Σύνολο εκλεκτόρων
Διαβάζω τα πλήρη αρχεία με το σύνολο των εκλεκτόρων που έχουν κατέβει σε excel
Νωρίτερα τα έχω μετατρέψει σε feather (σε άλλο ipynb) για να τα φορτώνω ταχύτερα

In [8]:
df_eklektores2024 = pd.read_feather(Path('../../files/mitroa/professors_export_202412111752.feather'))
df_eklektores2025 = pd.read_feather(Path('../../files/mitroa/professors_export_202510141042.feather'))

In [9]:
df_eklektores_previous = df_eklektores2024
df_eklektores_current = df_eklektores2025
# df_eklektores_previous
# df_eklektores_current
unique_eklektores_previous = list(df_eklektores_previous['Κωδικός Χρήστη'].unique().tolist())
unique_eklektores_current = list(df_eklektores_current['Κωδικός Χρήστη'].unique().tolist())
print(len(unique_eklektores_previous))
print(len(unique_eklektores_current))

18896
19328


### Γνωστικά αντικείμενα

#### Εισαγωγή από τα αρχεία excel

Διάβασμα γνωστικών αντικειμένων (field) και επιστημονικών πεδίων (domain) \
Το κάνω από προγενέστερο αρχείο excel με τα μητρώα των εξωτερικών εκλεκτόρων

Στη συνέχεια σώζω σε json αρχεία \
TODO: στην πορεία να το κοιτάξω να φορτώνω τα δεδομένα είτε από τα excel, είτε από τα json

In [10]:
excel_filename_2024 = '../../files/mitroa/Μητρώο Εξωτερικό 2024-12 ver4.xlsx'
fields_2024: dict[int, ScientificField] = get_full_scientific_fields_from_excel(excel_filename_2024)

excel_filename_2025 = '../../files/mitroa/Μητρώο Εξωτερικό 2025-10 ver3.xlsx'
fields_2025: dict[int, ScientificField] = get_full_scientific_fields_from_excel(excel_filename_2025)

#### Export json files

In [11]:
export_json_files = False

In [12]:
field_code = 555

pprint(fields_2024[field_code].info)
pprint(fields_2025[field_code].info)
if export_json_files:
    fields_2024[field_code].export_info_to_json(f'test-{field_code}-info-2024.json')
    fields_2025[field_code].export_info_to_json(f'test-{field_code}-info-2025.json')

ScientificFieldInfo(code=555, field_name='Πειραματική Μηχανική Δομικών Υλικών και Δομικών Στοιχείων', domain_name='Δομική Μηχανική', eklektores_idiou=[25804, 17939, 19760, 1588, 23150, 18206, 343, 18916], eklektores_synafous=[6099, 10800, 9503, 4764, 17547, 28213, 16057, 2910, 16998, 31610, 1089, 9772, 22815, 9739, 4538, 7834, 6223])
ScientificFieldInfo(code=555, field_name='Πειραματική Μηχανική Δομικών Υλικών και Δομικών Στοιχείων', domain_name='Δομική Μηχανική', eklektores_idiou=[25804, 2643, 15530, 3193, 19760, 42218, 1588, 23150, 18206, 343, 19441, 18916], eklektores_synafous=[39667, 6099, 10800, 9503, 4764, 17547, 28213, 16057, 2910, 16998, 31610, 1089, 9772, 22815, 9739, 4538, 7834, 6223])


In [13]:
if export_json_files:
    for field_code in fields_2024.keys():
        fields_2024[field_code].export_info_to_json(f'../../files/mitroa/json2024/{field_code}-info-2024.json')

In [14]:
if export_json_files:
    for field_code in fields_2025.keys():
        fields_2025[field_code].export_info_to_json(f'../../files/mitroa/json2025/{field_code}-info-2025.json')

## Έλεγχοι

In [15]:
field_code = 555
print(fields_2024[field_code].info.eklektores_idiou)
print(fields_2024[field_code].info.eklektores_synafous)

[25804, 17939, 19760, 1588, 23150, 18206, 343, 18916]
[6099, 10800, 9503, 4764, 17547, 28213, 16057, 2910, 16998, 31610, 1089, 9772, 22815, 9739, 4538, 7834, 6223]


In [16]:
def get_all_eklektores_in_year(year_fields: dict[int, ScientificField]) -> list:

    field_codes = list(year_fields.keys())

    # Collect all eklektores from all fields
    all_eklektores = []
    for field_code in field_codes:
        all_eklektores.extend(year_fields[field_code].info.eklektores_idiou)
        all_eklektores.extend(year_fields[field_code].info.eklektores_synafous)

    # Get unique values
    unique_eklektores = list(set(all_eklektores))
    print(f"Total unique eklektores: {len(unique_eklektores)}")
    return unique_eklektores

In [17]:
unique_eklektores_2024: list = get_all_eklektores_in_year(fields_2024)
unique_eklektores_2025: list = get_all_eklektores_in_year(fields_2025)
# unique_eklektores_2024

Total unique eklektores: 350
Total unique eklektores: 363


In [18]:
# Find eklektores that appear in 2024 but not in 2025
eklektores_only_in_2024 = list(set(unique_eklektores_2024) - set(unique_eklektores_2025))
print(f"Eklektores only in 2024: {len(eklektores_only_in_2024)}")
eklektores_only_in_2024

Eklektores only in 2024: 20


[18944,
 3586,
 17939,
 4505,
 17307,
 17701,
 9254,
 8104,
 18222,
 2610,
 3764,
 19001,
 19647,
 4557,
 18256,
 18268,
 17516,
 4984,
 5246,
 19583]

### Εκλέκτορες ανά γνωστικό αντικείμενο της προηγούμενης χρονιάς